In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

In [4]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 3.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
from wordcloud import WordCloud

In [6]:
train=pd.read_csv('twittertrain.csv')
test=pd.read_csv('twittertest.csv')

In [7]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
print(train.shape)
test.shape

(7613, 5)


(3263, 4)

In [9]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [10]:
train=train.drop(columns=['id','keyword','location'])
train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [11]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
feat=train[['text']]
label=train['target']

In [13]:
ps=PorterStemmer()
wordnet=WordNetLemmatizer()

In [14]:
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [15]:
def preprocessing(data):
    corpus=[]
    for i in range(len(data)):
        #remove urls
        tweet=re.sub(r'http\S+',' ',data[i])
        
        #remove html tags
        tweet=re.sub(r'<.*?>',' ',tweet)
        tweet=re.sub('@\w+([-.]\w+)*',' ',tweet)
        tweet=re.sub('&\w+([-.]\w+)*',' ',tweet)
        
        #remove digits
        tweet=re.sub(r'\d+',' ',tweet)
        
        #remove hashtags
        tweet=re.sub(r'#\w+',' ',tweet)
        #remove punctuations and numbers
        review=re.sub('[^a-zA-Z]',' ',tweet)
        review=review.lower()
        review=review.split()
        
        #review=expand_contractions(review)
        lemma=WordNetLemmatizer()
        review=[lemma.lemmatize(word) for word in review if word not in all_stopwords]
        review=' '.join(review)
        corpus.append(review)
    return corpus

In [16]:
feat.head()

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [17]:
feat['text']=preprocessing(feat['text'])

/var/folders/ft/nr64ylkn0lqb43gh8m5vvy3h0000gn/T/ipykernel_78026/386183970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat['text']=preprocessing(feat['text'])


In [18]:
feat.head()

,text
0,deed reason may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked shelter place notified officer ...
3,people receive evacuation order california
4,got sent photo ruby smoke pours school


In [19]:
#Model Building
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
#preprocessing the text data
test=test.drop(columns=['id','keyword','location'])
test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [21]:
test['text']=preprocessing(test['text'])
test.head()

,text
0,happened terrible car crash
1,heard different city stay safe everyone
2,forest fire spot pond goose fleeing across str...
3,apocalypse lighting
4,typhoon soudelor kill china taiwan


In [22]:
test.shape

(3263, 1)

In [41]:
!pip install --upgrade 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [45]:
!pip install gensim

In [53]:
!pip3 install gensim==3.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 15.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.6.0-cp39-cp39-macosx_10_9_x86_64.whl size=23341195 sha256=1f387ea053e43ebe7d9f48f005afada46df81a5ee918e1798b1833467f869a0c
  Stored in directory: /Users/xiaoqin/Library/Caches/pip/wheels/61/12/f2/84de20fba5e870553796b0834d11109992f06ddc20aaead086
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


In [61]:
!python3 -m pip install cryptography==38.0.4

In [63]:
import OpenSSL

AttributeError: module 'lib' has no attribute 'OpenSSL_add_all_algorithms'

In [62]:
import gensim

AttributeError: module 'lib' has no attribute 'OpenSSL_add_all_algorithms'

In [26]:
a=feat['text']
b=test['text']

In [27]:
sentence=pd.concat([a,b],axis=0)

In [28]:
sentence.head(5)

0                      deed reason may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3           people receive evacuation order california
4               got sent photo ruby smoke pours school
Name: text, dtype: object

In [29]:
print(len(sentence))

10876


In [30]:
print(type(sentence))
print(sentence.shape)
print(sentence)

<class 'pandas.core.series.Series'>
(10876,)
0                         deed reason may allah forgive u
1                   forest fire near la ronge sask canada
2       resident asked shelter place notified officer ...
3              people receive evacuation order california
4                  got sent photo ruby smoke pours school
                              ...                        
3258    earthquake safety los angeles safety fastener ...
3259    storm ri worse last hurricane city others hard...
3260                        green line derailment chicago
3261              meg issue hazardous weather outlook hwo
3262                   activated municipal emergency plan
Name: text, Length: 10876, dtype: object


In [31]:
sentence.iloc[1]

'forest fire near la ronge sask canada'

In [34]:
'forest' in sentence.iloc[1]

True

In [36]:
w2v_model = gensim.models.Word2Vec(sentence,min_count=1,vector_size=100)

NameError: name 'gensim' is not defined

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... /